In [5]:
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI


In [6]:


# Initialize Chroma retriever
embedding = OpenAIEmbeddings()
chroma_vectorstore = Chroma(persist_directory="chroma", embedding_function=embedding)
chroma_retriever = chroma_vectorstore.as_retriever(search_kwargs={"k": 3})


In [ ]:

# Initialize BM25 retriever
bm25_retriever = BM25Retriever.from_documents(splits)
bm25_retriever.k = 3  # Adjust as needed

# Initialize ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, chroma_retriever],
    weights=[0.5, 0.5]
)

# Set up the QA chain
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=ensemble_retriever,
    return_source_documents=True
)

# Example usage
query = "Your question here"
result = qa_chain({"query": query})
print(result['result'])